# More Results

In [1]:
import pandas as pd
import os
from scipy.stats import norm
from models.loss import crps_averaged
import torch

In [2]:
PATH = "trained_models"
MODELS = ["drn_24h", "drn_72h", "drn_120h"]
DATASETS = ["rf", "f"]

In [3]:
for m in MODELS:
    for d in DATASETS:
        preds = pd.read_csv(os.path.join(PATH, m, f"{d}_results.csv"))
        preds = preds.dropna()
        
        if d == "rf":
            conf = (11-1)/(11+1)
        elif d == "f":
            conf = (51-1)/(51+1)
        
        lower, upper = norm.interval(conf, loc=preds["mu"], scale=preds["sigma"])
        length = upper -lower
        preds["lower"] = lower
        preds["upper"] = upper
        preds["length"] = length
        preds["hit"] = (preds["lower"] <= preds["t2m"]) & (preds["upper"] >= preds["t2m"])
        
        preds.to_csv(os.path.join(PATH, m, f"{d}_results.csv"), index=False)
        
        mean_length = preds["length"].mean()
        coverage = preds["hit"].mean() * 100
        
        mu_sigma = torch.stack([torch.Tensor(preds.mu.values), torch.Tensor(preds.sigma.values)], dim=1)
        y = torch.Tensor(preds.t2m.values)
        crps = crps_averaged(mu_sigma, y).item()
        
        print(f"{m} {d} CRPS: {round(crps, 3)} & {round(mean_length, 3)} & {round(coverage, 3)}")
        
        #Log results
        with open(os.path.join(PATH, m, f"{d}.txt"), 'a') as file:
            file.write("\nMean PI length: " + str(preds["length"].mean()))
            file.write("\nCoverage: " + str(preds["hit"].mean()))

drn_24h rf CRPS: 0.65 & 2.79 & 78.381
drn_24h f CRPS: 0.614 & 4.263 & 94.873
drn_72h rf CRPS: 0.864 & 3.885 & 80.316
drn_72h f CRPS: 0.789 & 5.903 & 96.371
drn_120h rf CRPS: 1.187 & 5.273 & 79.527
drn_120h f CRPS: 1.106 & 7.989 & 95.821


## Show Model Summary

In [8]:
from utils.data import load_dataframes, load_distances, normalize_features_and_create_graphs, split_graph
import os
from torch_geometric.loader import DataLoader
from models.graphensemble.multigraph import Multigraph
import torch_geometric
from torch.optim import AdamW
import torch
import json
from dataclasses import dataclass
from torch_geometric.nn import summary
from torch_geometric.data import Data

if __name__ == "__main__":
    
    class Args:
        def __init__(self):
            self.data = "rf"
            self.leadtime = "24h"
    args = Args()
    
    print("#################################################")
    print(f"[INFO] Starting evaluation with data: {args.data} and leadtime: {args.leadtime}")
    print("#################################################")

    CHECKPOINT_FOLDER = f"trained_models/best_{args.leadtime}"
    JSONPATH = os.path.join(CHECKPOINT_FOLDER, "params.json")

    # Load the JSON file
    with open(JSONPATH, "r") as f:
        print(f"[INFO] Loading {JSONPATH}")
        args_dict = json.load(f)

    @dataclass
    class DummyConfig:
        pass

    for key, value in args_dict.items():
        setattr(DummyConfig, key, value)

    config = DummyConfig()
    print("[INFO] Starting eval with config: ", args_dict)

    # Dummy Data ####################################################################
    dummy_data =  Data(
            x=torch.ones((5, 36)),
            edge_index=torch.tensor([[0,1,2,3,4],[0,1,2,3,4]], dtype=torch.long),
            edge_attr=torch.tensor([1,2,3,4,5], dtype=torch.float),
            y=torch.tensor([1,2,3,4,5], dtype=torch.float),
            timestamp=None,
            n_idx=torch.tensor([0,1,2,3,4], dtype=torch.long)
        )
    dummy_test = [dummy_data for _ in range(100)]
    # Create Data Loaders ###########################################################
    print("[INFO] Creating data loaders...")
    train_loader = DataLoader(dummy_test, batch_size=config.batch_size, shuffle=True)

    # Create Model ##################################################################
    print("[INFO] Creating ensemble...")

    emb_dim = 20
    in_channels = 55  # graphs_train_rf[0].x.shape[1] + emb_dim - 1

    FOLDER = os.path.join(CHECKPOINT_FOLDER, "models")
    preds_list = []
    for path in os.listdir(FOLDER):
        if path.endswith(".ckpt"):
            print(f"[INFO] Loading model from {path}")
            # Load Model from chekcpoint
            #checkpoint = torch.load(os.path.join(FOLDER, path), map_location="cpu")
            print(f"[INFO] Loading model...")
            multigraph = Multigraph(
                embedding_dim=emb_dim,
                in_channels=in_channels,
                hidden_channels_gnn=config.gnn_hidden,
                out_channels_gnn=config.gnn_hidden,
                num_layers_gnn=config.gnn_layers,
                heads=config.heads,
                hidden_channels_deepset=config.gnn_hidden,
                optimizer_class=AdamW,
                optimizer_params=dict(lr=config.lr),
            )
            print(f"[INFO] Compiling model...")
            torch_geometric.compile(multigraph)

            print("[INFO] Dummy forward pass...")
            # run a dummy forward pass to initialize the model
            batch = next(iter(train_loader))
            batch = batch  # .to("cuda")
            multigraph  # .to("cuda")
            multigraph.forward(batch)

            print("[INFO] Summary...")
            #multigraph.load_state_dict(checkpoint["state_dict"])
            print(summary(model=multigraph, data=batch))
            break
            

#################################################
[INFO] Starting evaluation with data: rf and leadtime: 24h
#################################################
[INFO] Loading trained_models/best_24h/params.json
[INFO] Starting eval with config:  {'batch_size': 8, 'gnn_hidden': 265, 'gnn_layers': 2, 'heads': 8, 'lr': 0.0002, 'max_dist': 100, 'max_epochs': 31}
[INFO] Creating data loaders...
[INFO] Creating ensemble...
[INFO] Loading model from run_1.ckpt
[INFO] Loading model...
[INFO] Compiling model...
[INFO] Dummy forward pass...
[INFO] Summary...
+-----------------------------------------+---------------------------+----------------+------------+
| Layer                                   | Input Shape               | Output Shape   | #Param     |
|-----------------------------------------+---------------------------+----------------+------------|
| Multigraph                              |                           | [859, 2]       | 10,019,707 |
| ├─(encoder)EmbedStations            